## Extract, Transform and Load


* Create a `crude_db` database in pgAdmin 4 then create the following two tables within:

  * A `crude` table that contains the columns `Date`, `crude_open_price` and `crude_avg_price`.

  * Be sure to assign a primary key, as Pandas will not be able to do so.

* **Extraction**

  * Put each CSV into a pandas DataFrame.

* **Transform**

  * Copy only the columns needed into a new DataFrame.

  * Rename columns to fit the tables created in the database.


In [2]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

### Extract CSVs into DataFrames

In [3]:
#rad data file
crudeoil_file = "./Crude Oil WTI Futures Historical Data.csv"
crudeoil_df = pd.read_csv(crudeoil_file)
crudeoil_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,14-Oct-19,53.48,54.80,54.81,52.80,-,-2.41%
1,13-Oct-19,54.80,54.88,54.88,54.70,-,0.18%
2,11-Oct-19,54.70,53.88,54.93,53.64,741.70K,2.15%
3,10-Oct-19,53.55,52.69,53.97,51.38,613.94K,1.83%
4,9-Oct-19,52.59,52.57,53.74,52.31,632.67K,-0.08%


In [4]:
#identifying data types
crudeoil_df.dtypes

Date         object
Price       float64
Open        float64
High        float64
Low         float64
Vol.         object
Change %     object
dtype: object

### Transform premise DataFrame

In [5]:
#calculate the average price for the day using high and low prices
#add the new values into the column
crudeoil_df["crudeoil_avg_price"] = crudeoil_df[['High','Low']].mean(axis=1)
crudeoil_df.head()

,Date,Price,Open,High,Low,Vol.,Change %,crudeoil_avg_price
0,14-Oct-19,53.48,54.80,54.81,52.80,-,-2.41%,53.805
1,13-Oct-19,54.80,54.88,54.88,54.70,-,0.18%,54.790
2,11-Oct-19,54.70,53.88,54.93,53.64,741.70K,2.15%,54.285
3,10-Oct-19,53.55,52.69,53.97,51.38,613.94K,1.83%,52.675
4,9-Oct-19,52.59,52.57,53.74,52.31,632.67K,-0.08%,53.025


In [6]:
#format date column into date type
crudeoil_df['Date'] =  pd.to_datetime(crudeoil_df['Date'],format='%d-%b-%y')
crudeoil_df['Date']

0      2019-10-14
1      2019-10-13
2      2019-10-11
3      2019-10-10
4      2019-10-09
          ...    
2531   2010-01-08
2532   2010-01-07
2533   2010-01-06
2534   2010-01-05
2535   2010-01-04
Name: Date, Length: 2536, dtype: datetime64[ns]

In [7]:
crudeoil_df.dtypes

Date                  datetime64[ns]
Price                        float64
Open                         float64
High                         float64
Low                          float64
Vol.                          object
Change %                      object
crudeoil_avg_price           float64
dtype: object

In [8]:
# Create a filtered dataframe from specific columns
crudeoil_cols = ["Date", "Open","crudeoil_avg_price"]
crudeoil_transformed= crudeoil_df[crudeoil_cols].copy()

In [9]:
# Rename the column headers
crudeoil_transformed = crudeoil_transformed.rename(columns={"Open": "crude_open_price"})

# Clean the data by dropping duplicates and setting the index
# crudeoil_transformed.drop_duplicates("id", inplace=True)
#crudeoil_transformed.set_index("Date", inplace=True)

crudeoil_transformed.head()

,Date,crude_open_price,crudeoil_avg_price
0,2019-10-14,54.80,53.805
1,2019-10-13,54.88,54.790
2,2019-10-11,53.88,54.285
3,2019-10-10,52.69,52.675
4,2019-10-09,52.57,53.025


In [10]:
#format crudeoil_avg_price to two decimal place
crudeoil_transformed=crudeoil_transformed.round({'crudeoil_avg_price': 2})
crudeoil_transformed.head()

,Date,crude_open_price,crudeoil_avg_price
0,2019-10-14,54.80,53.80
1,2019-10-13,54.88,54.79
2,2019-10-11,53.88,54.28
3,2019-10-10,52.69,52.68
4,2019-10-09,52.57,53.03


In [16]:
crudeoil_transformed.to_csv('../../output/crude_oil_transformed.csv', index=False)